<a href="https://colab.research.google.com/github/ibraztech2/VisionMarshal/blob/main/vision_marshal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  ***Backend Setup***

In [1]:
!rm -rf /content/smart_traffic_detector

## `1. Drive Mount`

In [2]:
# Check the version of cuda compiler available
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [3]:
%%time
#Mount drive to get acces to video input

#from google.colab import drive
#drive.mount('/content/drive')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


## `2. MMAction2 installation And Dependencies Resolution`

### 1. Downgrade Pytorch, Numpy, Transformers, Torchvision, Torchaudio to compatible Version


In [ ]:
%%time

# !pip uninstall -y torch torchvision torchaudio mmcv mmcv-full mmengine numpy torchao transformers
!pip uninstall -y torch torchvision torchaudio numpy transformers


# Install PyTorch 2.1.0 with CUDA 11.8
!pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 657.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB

### 2. Install MMCV and MMengine

In [ ]:
%%time

# Install MMCV 2.1.0 and mmengine
!pip install mmcv==2.1.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html
!pip install mmengine==0.10.3




### 3. Install OpenMIM ( command line tool for MMAction and  MMdetection )

In [ ]:
%%time

# Install OpenMIM
!pip install -U openmim

### 4. Build MMaction   from source

In [ ]:
# Clone MMAction from Github

!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2
!pip install -e .



### 5. Install Numpy compatible Version

In [ ]:
!pip install numpy==1.24.4


### 6. Download  Temporal Segment Network ( Built on Imagenet ) configuration file and  checkpoint file

In [ ]:
# downloading the configuration file to the given location

!mim download mmaction2 --config tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb --dest /content/mmaction2/


### 7. Testing Setup for :

#### 1.  MMAction Setup on a demo video

In [ ]:
# The demo.mp4 and label_map_k400.txt are both from Kinetics-400
!python /content/mmaction2/demo/demo.py /content/mmaction2/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb.py \
    /content/mmaction2/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth \
    /content/mmaction2/demo/demo.mp4 /content/mmaction2/tools/data/kinetics/label_map_k400.txt

#### 2. MMDetcttion on a Demo picture

In [ ]:
!python /content/mmdetection/demo/image_demo.py /content/mmdetection/demo/demo.jpg /content/mmdetection/rtmdet_tiny_8xb32-300e_coco.py --weights /content/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth --device cpu

## `3.  App File, Requirement and Checkpoint setup `

### 1. Download Checkpoints  Files for Action recognition [Timesformer](https://mmaction2.readthedocs.io/en/latest/model_zoo/recognition.html#timesformer)

In [ ]:
# Create Project  Tree Struscture
import os

os.makedirs("/content/smart_traffic_detector", exist_ok=True)

os.makedirs("/content/smart_traffic_detector/debug", exist_ok=True)
os.makedirs("/content/smart_traffic_detector/main", exist_ok=True)
os.makedirs("/content/smart_traffic_detector/checkpoints", exist_ok=True)
os.makedirs("/content/smart_traffic_detector/clips", exist_ok=True)
os.makedirs("/content/smart_traffic_detector/violations", exist_ok=True)
os.makedirs("/content/smart_traffic_detector/alerts", exist_ok=True)

In [ ]:

# Download Model  Checkpoint  for ACtion Recognition

!wget https://download.openmmlab.com/mmaction/v1.0/recognition/timesformer/timesformer_spaceOnly_8xb8-8x32x1-15e_kinetics400-rgb/timesformer_spaceOnly_8xb8-8x32x1-15e_kinetics400-rgb_20220815-78f05367.pth -P /content/smart_traffic_detector/checkpoints
#!wget https://download.openmmlab.com/mmaction/v1.0/recognition/slowfast/slowfast_r101_8xb8-8x8x1-256e_kinetics400-rgb/slowfast_r101_8xb8-8x8x1-256e_kinetics400-rgb_20220818-9c0e09bd.pth -P /content/smart_traffic_detector/checkpoints
#!wget https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth -P /content/smart_traffic_detector/checkpoints
!wget https://github.com/open-mmlab/mmaction2/blob/main/tools/data/kinetics/label_map_k400.txt -P /content/smart_traffic_detector

### 3. Write App file and  Requirements file

#### 1. write Requirements File

In [ ]:
%%writefile requirements.txt


# Core video processing
moviepy==1.0.3
opencv-python==4.9.0.80
ffmpeg-python==0.2.0

# Object detection (YOLOv8)
ultralytics==8.1.5


# Email alerts
certifi
email-validator

# Streamlit (optional for dashboard)
streamlit==1.27.2

# For parsing configs and data
PyYAML==6.0
numpy==1.24.4
pandas

# Youtube  video library
yt-dlp
pyngrok



# frontend and tunnel library
pyngrok
streamlit
ultralytics

#### 2. write App file

In [ ]:
%%writefile /content/smart_traffic_detector/main/main_app.py
import os
import json
import subprocess
from typing import Dict, List, Any, Set

import cv2
import numpy as np
from moviepy.editor import VideoFileClip
from mmaction.apis import init_recognizer, inference_recognizer
from ultralytics import YOLO




#--------------------------------------------
# Environment Project Tree Structure Setup
#--------------------------------------------

work_dir = "/content/smart_traffic_detector"
model_config = "/content/mmaction2/configs/recognition/timesformer/timesformer_spaceOnly_8xb8-8x32x1-15e_kinetics400-rgb.py"
model_checkpoints = "/content/smart_traffic_detector/checkpoints/timesformer_spaceOnly_8xb8-8x32x1-15e_kinetics400-rgb_20220815-78f05367.pth"
label_map_path = "/content/smart_traffic_detector/label_map_k400.txt"




# ------------------------------------------------------------------
# Video Preprocessing
# ------------------------------------------------------------------
def split_video_into_clips(video_path: str, output_dir: str = "clips", clip_length: int = 10):
    """Split a  video into tubelets  (smaller clips (seconds))  via ffmpeg."""


    # Probe duration
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-show_entries", "format=duration",
         "-of", "default=noprint_wrappers=1:nokey=1", video_path],
        capture_output=True, text=True
    )
    duration = float(result.stdout.strip()) if result.stdout.strip() else 0.0

    clips = []  # temorarily store splitted clips
    for start in range(0, int(duration), clip_length):  # iterate over the clip length at a step size of 10sec
        clip_name = f"{output_dir}/clip_{start:04d}.mp4"
        subprocess.run([
            "ffmpeg", "-y",
            "-ss", str(start),
            "-i", video_path,
            "-t", str(clip_length),
            "-vf", "scale=640:384",
            "-c:v", "libx264",
            "-pix_fmt", "yuv420p",
            clip_name
        ], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
        clips.append((clip_name, start))
    return clips


# ------------------------------
# Traffic Light Color Detection
# ------------------------------

def detect_traffic_light_color_simple(image, bbox, debug_id=None):
    """

    Returns: 'red', 'yellow', 'green', or 'unknown'.
    """
    x1, y1, x2, y2 = map(int, bbox)

    # Clip to image bounds
    h_img, w_img = image.shape[:2]
    x1 = max(0, min(x1, w_img - 1))
    x2 = max(0, min(x2, w_img))
    y1 = max(0, min(y1, h_img - 1))
    y2 = max(0, min(y2, h_img))
    if x2 <= x1 or y2 <= y1:
        return "unknown"

    roi = image[y1:y2, x1:x2]  # Grab the region of interest from the Detected Traffic Light
    if roi.size == 0:
        return "unknown"

    # Noise reduction
    roi = cv2.GaussianBlur(roi, (3, 3), 0)

    # increase contrast
    lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    roi = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # HSV  (Hue Saturation Value)
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Split into 3 horizontal sections: Red Yellow and Green which falls in top, Mid and bottom of the traffic light respectively

    h = roi.shape[0]
    third = max(1, h // 3)
    sections = {                     # Grab  each section and store them based on the division
        "red": hsv[0:third, :],
        "yellow": hsv[third:2 * third, :],
        "green": hsv[2 * third:h, :]
    }

    # Broad HSV ranges (tune per camera)
    red_lower1 = np.array([0, 50, 20])
    red_upper1 = np.array([10, 255, 255])
    red_lower2 = np.array([160, 50, 20])
    red_upper2 = np.array([180, 255, 255])

    yellow_lower = np.array([15, 40, 40])
    yellow_upper = np.array([40, 255, 255])

    green_lower = np.array([30, 40, 40])
    green_upper = np.array([100, 255, 255])

     # Debbug traffic light section to detetct the correct light
    scores = {}
    if debug_id:
        dbg_dir = os.path.join(work_dir, "debug")
        os.makedirs(dbg_dir, exist_ok=True)
        cv2.imwrite(os.path.join(dbg_dir, f"tl_{debug_id}.jpg"), roi)
    else:
        dbg_dir = None


    # masking ROI to get values the approximate shape and the location of the color detected
    for color, section_hsv in sections.items():
        if color == "red":
            mask1 = cv2.inRange(section_hsv, red_lower1, red_upper1)
            mask2 = cv2.inRange(section_hsv, red_lower2, red_upper2)
            mask = cv2.bitwise_or(mask1, mask2)
        elif color == "yellow":
            mask = cv2.inRange(section_hsv, yellow_lower, yellow_upper)
        else:  # green
            mask = cv2.inRange(section_hsv, green_lower, green_upper)

        scores[color] = cv2.countNonZero(mask)  # count the number of the dominant color of each pixel in each section of the traffic light

        if dbg_dir:
            cv2.imwrite(os.path.join(dbg_dir, f"{color}_mask_{debug_id}.jpg"), mask)   # store Detected color

    dominant_color = max(scores, key=scores.get)
    total_pixels = roi.shape[0] * roi.shape[1]
    return dominant_color if total_pixels > 0 and (scores[dominant_color] / float(total_pixels)) > 0.002 else "unknown"


# ------------------------------
# Object Detection using Yolov8
# ------------------------------
_object_model = YOLO("yolov8n.pt")  # pretrained COCO

 # Return Respective classs id for the object detcted
if isinstance(_object_model.names, dict):
    CLASS_NAMES: Dict[int, str] = _object_model.names
elif isinstance(_object_model.names, (list, tuple)):
    CLASS_NAMES = {i: n for i, n in enumerate(_object_model.names)}
else:
    CLASS_NAMES = {}


def _get_cls_id(box) -> int:
    cid = getattr(box, "cls", None)
    if cid is None:
        return -1
    try:
        return int(cid.item())
    except Exception:
        try:
            return int(cid[0])
        except Exception:
            return int(cid)


def run_object_detection(clip_path: str) -> List[Dict[str, Any]]:
    """Run YOLO object detection on a video file and return list of detections."""
    results = _object_model(clip_path, stream=True)
    detections = []

    for frame_id, res in enumerate(results):
        frame = res.orig_img  # BGR numpy array

        boxes = getattr(res, "boxes", None)
        if boxes is None or len(boxes) == 0:
            continue

        for box_idx, box in enumerate(boxes):
            cls_id = _get_cls_id(box)
            cls_name = CLASS_NAMES.get(cls_id, f"class_{cls_id}")

            # get bounding box cordinate
            xyxy = box.xyxy
            if hasattr(xyxy, "cpu"):
                xyxy = xyxy.cpu().numpy()
            xyxy = np.array(xyxy).reshape(-1, 4)[0].tolist()

            # confidence check
            conf_attr = getattr(box, "conf", None)
            if conf_attr is not None:
                try:
                    conf_val = float(conf_attr.item())
                except Exception:
                    conf_val = float(conf_attr)
            else:
                conf_val = 0.0

            det = {
                "frame": frame_id,
                "class": cls_name,
                "confidence": conf_val,
                "bbox": xyxy,
            }

            # Traffic light color classification
            if cls_name == "traffic light":
                debug_id = f"{os.path.basename(clip_path)}_f{frame_id}_b{box_idx}"
                det["color"] = detect_traffic_light_color_simple(frame, xyxy, debug_id=debug_id)

            detections.append(det)

    return detections

#--------------
# Model Loader
#-------------
def load_model():
  _object_model = YOLO("yolov8n.pt")
  _action_model = init_recognizer(model_config, model_checkpoints, device="cpu")
  return _object_model, _action_model

_object_model, _action_model = load_model()

# ----------------------------------------------------
# Action Recognition  Using TimeSformer from MMAction
# ----------------------------------------------------


with open(label_map_path) as f:
    LABEL_MAP = [line.strip() for line in f.readlines()]


def run_action_recognition(clip_path: str) -> List[Dict[str, Any]]:
    """Return top-5 action predictions from MMAction2 model."""
    results = inference_recognizer(_action_model, clip_path)
    top5_scores, top5_labels = results.pred_score.topk(5)
    return [
        {"label": LABEL_MAP[int(label)], "score": float(score)}
        for label, score in zip(top5_labels, top5_scores)
    ]

# ----------------------
# clip Analysis
# ----------------------
def analyze_clip(clip_path: str, start_time: float):
    obj_dets = run_object_detection(clip_path)
    act_dets = run_action_recognition(clip_path)
    return {
        "clip": os.path.basename(clip_path),
        "start_time": start_time,
        "object_detections": obj_dets,
        "action_detections": act_dets
    }


# -------------------------------------------
# Violation Rules for both Action and Object
# -------------------------------------------
violation_rules = {
    "driving car": {
        "required_objects": ["traffic light", "car"],
        "threshold": 0.05,
        "description": "🚨 RED LIGHT VIOLATION Detected. "
    },
    "riding a bike": {
        "required_objects": ["bicycle", "person"],
        "threshold": 0.25,
        "description": "Bike riding – check for helmet use"
    },
    "motorcycling": {
        "required_objects": ["motorcycle", "person"],
        "threshold": 0.25,
        "description": "Motorcycle detected – possible no helmet"
    },
    "running": {
        "required_objects": ["person", "car"],  #
        "threshold": 0.30,
        "description": "Jaywalking or unsafe crossing"
    },
    "sitting": {
        "required_objects": ["person", "bus stop"],
        "threshold": 0.15,
        "description": "Loitering at bus stop"
    },
    "standing": {
        "required_objects": ["person"],
        "threshold": 0.20,
        "description": "Pedestrian blocking road"
    },
}

# Map logical -> detector labels (yolov8 was trained on COCO with 80 objects)
OBJECT_SYNONYMS = {
    "bus stop": ["bench", "stop sign"],
}


def _expand_required_objects(required: Set[str], detected_objects: Set[str]) -> Set[str]:
    """Expand logical rule objects into actual detector class names where needed."""
    expanded = set()
    for obj_name in required:
        if obj_name in detected_objects:
            expanded.add(obj_name)
            continue
        for syn in OBJECT_SYNONYMS.get(obj_name, []):
            if syn in detected_objects:
                expanded.add(syn)
                break
        else:
            expanded.add(obj_name)  # keep; may remain missing
    return expanded


# --------------------------
# Violation Detection Logic
# -------------------------
def detect_violations(result: Dict[str, Any]):
    violations = []
    obj_dets = result["object_detections"]
    detected_objects = set(obj["class"] for obj in obj_dets)

    # Aggregate traffic-light colors (any RED beats others)
    tl_colors = [obj.get("color", "unknown") for obj in obj_dets if obj["class"] == "traffic light"]
    traffic_light_status = "unknown"

    # Priority List of colors
    if any(c == "red" for c in tl_colors):
        traffic_light_status = "red"
    elif any(c == "yellow" for c in tl_colors):
        traffic_light_status = "yellow"
    elif any(c == "green" for c in tl_colors):
        traffic_light_status = "green"

    for action in result["action_detections"]:
        label, score = action["label"], action["score"]
        if label not in violation_rules:
            continue
        rule = violation_rules[label]

        if score < rule["threshold"]:
            print(f"⚠️ Action '{label}' below threshold: {score:.2f} < {rule['threshold']}")
            continue

        logical_required = set(rule["required_objects"])
        required = _expand_required_objects(logical_required, detected_objects)
        missing = {r for r in required if r not in detected_objects}



        #  Rule-based violation Description
        if not missing and traffic_light_status == "red":
            violations.append({
                    "clip": result["clip"],
                    "start_time": result["start_time"],
                    "violation_type": "Red Light Violation",
                    "description": "Vehicle crossed / moved during RED light.",
                    "score": score,
                    "objects": list(logical_required),
                    "light": traffic_light_status
                })
            print(f"⚠️ Violation Detected: {label}.")
        else:
            print(f"❌ Missing required objects for {label}: {missing}")

    return violations


# -----------------------------------------
# Run Violation Engine on All Clip Results
# ----------------------------------------
def run_violation_engine(processed_results: List[Dict[str, Any]]):
    all_violations = []
    for result in processed_results:
        v = detect_violations(result)
        if v:
            all_violations.extend(v)
    return all_violations


# --------------------------------
# Extract Violation Evidence Clip
# --------------------------------
def extract_violation_clip(input_path: str, output_path: str, start: float = 0, clip_duration: int = 5):
    video = VideoFileClip(input_path)
    video_duration = video.duration

    # Clamp start
    if start >= video_duration:
        start = max(0, video_duration - clip_duration)

    end = min(start + clip_duration, video_duration)
    print(f"Extracting from {start:.2f}s to {end:.2f}s (clip is {video_duration:.2f}s long)")

    clip = video.subclip(start, end)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    clip.write_videofile(output_path, codec="libx264", audio_codec="aac")
    clip.close()
    video.close()


# -----------------
# Email Alert Stub
# ------------------
def send_violation_email():
    # Implement Sending Email Algorithm that report violators to authority and may also send automated alert with fine to the violator but with license plate reognition
    pass


# ---------------
# Main Funtion
# ---------------
def main_func(video_path: str):

    #  Preprocess video into tubelet
    clips = split_video_into_clips(video_path, output_dir=os.path.join(work_dir,"clips"))
    processed_results = []

    # Detect objects + actions per tubelet
    for clip_path, start in clips:
        print(f"Analyzing {clip_path} (start at {start}s)...")
        result = analyze_clip(clip_path, start)
        processed_results.append(result)

    # Save analysed reult
    analysed_path = os.path.join(work_dir, "analysed_results.json")
    with open(analysed_path, "w") as f:
        json.dump(processed_results, f, indent=2)
    print(f"Analysis saved: {analysed_path}")

    # Check  for vioaltions
    violations = run_violation_engine(processed_results)

    # Extract evidence clips
    for violation in violations:
        clip = violation["clip"]
        start_time = violation["start_time"]
        violation_type = violation["violation_type"]
        violation_clip_name = os.path.join(work_dir, f"clips/{violation_type.replace(' ', '_').lower()}_{clip}")
        print(f"Saving violation clip as: {violation_clip_name}")
        input_clip_path = f"/content/smart_traffic_detector/clips/{clip}"
        extract_violation_clip(input_clip_path, violation_clip_name, start_time, clip_duration=5)

    # Save violation log
    violation_log_path = os.path.join(work_dir,"violations", "violation_log.json")
    with open(violation_log_path, "w") as f:
        json.dump(violations, f, indent=2)
    print(f"Violation log saved: {violation_log_path}")

    print("Pipeline complete.")
    return processed_results, violations


# ----------
# CLI Entry
# ----------

if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description="Smart Traffic Violation Detection Pipeline")
    parser.add_argument("--video", type=str, required=True, help="Path to the input video file")
    args = parser.parse_args()
    main_func(args.video)


#### 4. Write Streamlit App

In [ ]:
%%writefile /content/smart_traffic_detector/streamlit_app.py

from pathlib import Path
import tempfile
import os
import json
import streamlit as st
from main.main_app import main_func

# -------------------------------
# Streamlit Page Configuration
# -------------------------------
st.set_page_config(
    page_title="🚦 Smart Traffic Violation Detector",
    page_icon="🚨",
    layout="centered"
)

# -------------------------------
# Sidebar Info
# -------------------------------
st.sidebar.title("📖 How to use")
st.sidebar.markdown("""
1. Upload a traffic video (MP4, AVI, MOV, WEBM).
2. Click **'🚨 Check Violations'** to analyze the video.
3. Review detected violations and download logs/clips.
""")
st.sidebar.info("This tool uses AI to detect red-light running, jaywalking, and helmetless riding and other Infractions.")

# -------------------------------
# Main Title
# -------------------------------
st.title("🚦 Smart Traffic Violation Detector")
st.markdown("""
Upload a traffic video to detect potential violations like **red-light running**,
**jaywalking**, or **helmetless riding**.
""")

# -------------------------------
# File Uploader
# -------------------------------
uploaded_video = st.file_uploader(
    "📤 Upload a video file to analyze",
    type=["mp4", "avi", "mov", "webm"]
)

# Initialize session state
if "violations" not in st.session_state:
    st.session_state["violations"] = None
    st.session_state["log_ready"] = False

# -------------------------------
# Main Logic
# -------------------------------
if uploaded_video:
    st.video(uploaded_video)

    # Save uploaded video to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_video:
        temp_video.write(uploaded_video.read())
        temp_video_path = temp_video.name

    if st.button("🚨 Check Violations"):
        st.info("⚙️ Starting analysis... please wait.")
        with st.spinner("Analyzing video for traffic violations..."):
            try:
                processed_result, violations = main_func(temp_video_path)

                # Save results to session state
                st.session_state["violations"] = violations
                st.session_state["log_ready"] = True

                if violations:
                    st.success(f"✅ Analysis complete: {len(violations)} violation(s) detected.")
                else:
                    st.success("✅ No violations detected in the video.")

            except Exception as e:
                st.error(f"❌ An error occurred during analysis:\n\n{e}")

            finally:
                # Clean up temporary video file
                os.remove(temp_video_path)

    # -------------------------------
    # Display Results (if any)
    # -------------------------------
    if st.session_state["log_ready"]:
        violations = st.session_state["violations"]

        # Download violation log
        log_json = json.dumps(violations, indent=2)
        st.download_button(
            label="⬇️ Download Violation Log",
            data=log_json,
            file_name="violation_log.json",
            mime="application/json"
        )

        # log inline
        if st.checkbox("📄 View Violation Log"):
            st.code(log_json, language="json")

        # Show detected violation clips
        if violations:
            st.subheader("📹 Violation Clips")
            for i, violation in enumerate(violations):
                st.markdown(f"---\n### 🚨 Violation {i+1}: {violation['violation_type']}")
                st.write(f"**Description**: {violation['description']}")
                st.write(f"**Score**: {violation['score']:.2f}")

                clip_path = os.path.join(
                    "/content/smart_traffic_detector/clips", violation["clip"]
                )

                if os.path.exists(clip_path):
                    st.video(clip_path)
                    with open(clip_path, "rb") as clip_file:
                        st.download_button(
                            label=f"⬇️ Download Clip {i+1}",
                            data=clip_file.read(),
                            file_name=violation["clip"],
                            mime="video/mp4",
                            key=f"download_clip_{i}"
                        )
                else:
                    st.warning(f"⚠️ Clip not found: {violation['clip']}")

else:
    st.info("👆 Upload a video file above to get started.")


#### Write ` __init__file`

In [ ]:
%%writefile /content/smart_traffic_detector/main/__init__.py

from .main_app import main_func



In [ ]:
!rm -rf __pycache__ main/__pycache__


#### 4. Install Requirements

In [ ]:
!pip install --q -r requirements.txt

In [ ]:
!cp /content/mmaction2/tools/data/kinetics/label_map_k400.txt /content/smart_traffic_detector

## `4. Local Testing`

#### 1. Download Youtube Video

In [ ]:
!yt-dlp "https://youtu.be/lBRWEeuuMpE?si=AKzynUYzr0CcVQCq" -o /content/smart_traffic_detector/test_video_2


#### 2. Local Inferencing

In [ ]:

 !python /content/smart_traffic_detector/main/main_app.py  --video /content/smart_traffic_detector/test_video_2.webm

#  ***Frontend Setup***

#### 1. Get Local tunnel Password

In [ ]:
!curl ifconfig.me

34.16.215.240

#### 2. Deploy Streamlit Prototype

In [ ]:
!streamlit run /content/smart_traffic_detector/streamlit_app.py & npx localtunnel --port 8501




⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.215.240:8501

⠼⠴⠦⠧⠇  Stopping...
^C
